In [1]:
# ============================================================
# 📘 MACHINE LEARNING ENHANCEMENT FOR WATERMARK VERIFICATION
# ============================================================

import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
import cv2, json, random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import tensorflow as tf
from tensorflow.keras import layers, models
import seaborn as sns
from skimage.metrics import structural_similarity as ssim

# ============================================================
# 🔧 Minimal Watermarking Core (simplified for ML pipeline)
# ============================================================

class SimpleWatermarkSystem:
    def __init__(self):
        pass

    def preprocess_image(self, image_bytesio):
        image = Image.open(image_bytesio).convert('L')
        image = np.array(image.resize((128, 128))).astype(np.float32) / 255.0
        return image

    def calculate_complexity(self, image):
        gx, gy = np.gradient(image)
        return np.mean(np.sqrt(gx**2 + gy**2))

    def adaptive_alpha(self, complexity):
        return 0.05 + 0.3 * complexity

    def text_to_watermark(self, text, size=(32, 32)):
        bits = np.frombuffer(text.encode('utf-8'), dtype=np.uint8)
        bits = np.unpackbits(bits)[:size[0]*size[1]]
        return bits.reshape(size).astype(np.float32)

    def embed_watermark(self, host, watermark, alpha):
        host_f = cv2.dct(host)
        wm_resized = cv2.resize(watermark, host_f.shape)
        watermarked_f = host_f + alpha * wm_resized
        watermarked = cv2.idct(watermarked_f)
        return np.clip(watermarked, 0, 1)

    def extract_features(self, original, tampered):
        complexity = self.calculate_complexity(original)
        alpha = self.adaptive_alpha(complexity)
        ssim_val = ssim(original, tampered)
        mse = np.mean((original - tampered)**2)
        psnr = 10 * np.log10(1.0 / (mse + 1e-12))
        corr = np.corrcoef(original.flatten(), tampered.flatten())[0,1]
        hist_o, _ = np.histogram(original, bins=32, range=(0,1), density=True)
        hist_t, _ = np.histogram(tampered, bins=32, range=(0,1), density=True)
        hist_dist = np.sum(np.abs(hist_o - hist_t))
        return np.array([complexity, alpha, ssim_val, mse, psnr, corr, hist_dist], dtype=np.float32)


# ============================================================
# 🧪 Dataset Generation (synthetic images)
# ============================================================

def generate_synthetic_image(size=128):
    """Generate simple grayscale patterns."""
    img = np.zeros((size, size), dtype=np.float32)
    shape_type = random.choice(['circle','square','gradient'])
    if shape_type == 'circle':
        cv2.circle(img, (64,64), 30, random.uniform(0.4,0.9), -1)
    elif shape_type == 'square':
        cv2.rectangle(img, (32,32), (96,96), random.uniform(0.4,0.9), -1)
    else:
        img = np.tile(np.linspace(0,1,size), (size,1))
    return img

def apply_random_tamper(image):
    tampered = image.copy()
    tamper_type = random.choice(['noise','blur','crop','bright','contrast'])
    if tamper_type == 'noise':
        tampered += np.random.normal(0,0.05,tampered.shape)
    elif tamper_type == 'blur':
        tampered = cv2.GaussianBlur(tampered, (5,5), 1.0)
    elif tamper_type == 'crop':
        tampered[40:60,40:60] = 0.5
    elif tamper_type == 'bright':
        tampered = np.clip(tampered*1.2,0,1)
    elif tamper_type == 'contrast':
        tampered = np.clip((tampered-0.5)*1.3 + 0.5,0,1)
    return np.clip(tampered,0,1)

def create_dataset(system, n_samples=300):
    X, y = [], []
    for _ in range(n_samples):
        host = generate_synthetic_image()
        wm = system.text_to_watermark("HELLO", (32,32))
        alpha = system.adaptive_alpha(system.calculate_complexity(host))
        watermarked = system.embed_watermark(host, wm, alpha)
        # Authentic (label 1)
        features_auth = system.extract_features(host, watermarked)
        X.append(features_auth); y.append(1)
        # Tampered (label 0)
        tampered = apply_random_tamper(watermarked)
        features_tamp = system.extract_features(host, tampered)
        X.append(features_tamp); y.append(0)
    return np.array(X), np.array(y)


# ============================================================
# 🧠 ML Model Training & Evaluation
# ============================================================

system = SimpleWatermarkSystem()
X, y = create_dataset(system, n_samples=400)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

model = models.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=25, batch_size=16, validation_data=(X_val, y_val), verbose=1)

# ============================================================
# 📊 Results and Plots
# ============================================================

# --- Accuracy and Loss curves ---
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title("Model Accuracy"); plt.xlabel("Epoch"); plt.ylabel("Accuracy"); plt.legend()
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title("Model Loss"); plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.legend()
plt.show()

# --- Predictions ---
y_pred_prob = model.predict(X_val).flatten()
y_pred = (y_pred_prob > 0.5).astype(int)

# --- Confusion Matrix ---
cm = confusion_matrix(y_val, y_pred)
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Tampered','Authentic'], yticklabels=['Tampered','Authentic'])
plt.xlabel("Predicted"); plt.ylabel("True"); plt.title("Confusion Matrix")
plt.show()

# --- Classification Report ---
print("\nClassification Report:\n")
print(classification_report(y_val, y_pred, target_names=['Tampered','Authentic']))

# --- ROC Curve ---
fpr, tpr, _ = roc_curve(y_val, y_pred_prob)
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.3f})')
plt.plot([0,1], [0,1], 'k--')
plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

print(f"✅ Final Validation Accuracy: {np.mean(y_pred == y_val)*100:.2f}%")
print(f"✅ AUC Score: {roc_auc:.3f}")

# Save model for later
model.save("authenticity_classifier.h5")
print("\n💾 Model saved as authenticity_classifier.h5")


ModuleNotFoundError: No module named 'tensorflow'